In [ ]:
!pip install timm
!pip install torchvision --upgrade
!mkdir StandfordDog

     |████████████████████████████████| 431 kB 15.1 MB/s 
     |████████████████████████████████| 21.0 MB 14.8 MB/s 
     |████████████████████████████████| 750.6 MB 9.9 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.11.0 which is incompatible.


In [ ]:
from functools import partial
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from timm.models.layers import trunc_normal_, DropPath
from timm.models.registry import register_model
import tarfile
from Dataset import *
from prepare import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with tarfile.open('/content/drive/MyDrive/FGVC-project/CUB_200_2011.tgz', 'r:gz') as tar:
  tar.extractall(path='/content')

#with tarfile.open('/content/drive/MyDrive/FGVC-project/StandfordDog/images.tar', 'r') as tar:
  #tar.extractall(path='/content/StandfordDog')

#with tarfile.open('/content/drive/MyDrive/FGVC-project/StandfordDog/lists.tar', 'r') as tar:
  #tar.extractall(path='/content/StandfordDog')

In [ ]:
batch_size = 64
epochs=50
lr = 0.03
weight_decay = 1e-3
label_smoothing = 0.0

In [ ]:
train_transforms = transforms.Compose([transforms.Resize((256, 256), Image.BILINEAR),
                                       transforms.RandomCrop((224, 224)),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

test_transforms = transforms.Compose([transforms.Resize((256, 256), Image.BILINEAR),
                                        transforms.CenterCrop((224, 224)),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])


train_dataset = CUB2011(root='/content/', transform=train_transforms, train=True, extract=False)
test_dataset = CUB2011(root='/content/', transform=test_transforms, train=False, extract=False)


train_loader = DataLoader(train_dataset, batch_size=batch_size, 
                                              shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset, batch_size=batch_size, 
                                          shuffle=False, num_workers=1)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:333: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "


convNext base

In [ ]:
class ConvNeXt(nn.Module):
    r""" ConvNeXt
        A PyTorch impl of : `A ConvNet for the 2020s`  -
          https://arxiv.org/pdf/2201.03545.pdf
    Args:
        in_chans (int): Number of input image channels. Default: 3
        num_classes (int): Number of classes for classification head. Default: 1000
        depths (tuple(int)): Number of blocks at each stage. Default: [3, 3, 9, 3]
        dims (int): Feature dimension at each stage. Default: [96, 192, 384, 768]
        drop_path_rate (float): Stochastic depth rate. Default: 0.
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
        head_init_scale (float): Init scaling value for classifier weights and biases. Default: 1.
    """
    def __init__(self, in_chans=3, num_classes=1000, 
                 depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], drop_path_rate=0., 
                 layer_scale_init_value=1e-6, head_init_scale=1.,
                 ):
        super().__init__()

        self.downsample_layers = nn.ModuleList() # stem and 3 intermediate downsampling conv layers
        stem = nn.Sequential(
            nn.Conv2d(in_chans, dims[0], kernel_size=4, stride=4),
            LayerNorm(dims[0], eps=1e-6, data_format="channels_first")
        )
        self.downsample_layers.append(stem)
        for i in range(3):
            downsample_layer = nn.Sequential(
                    LayerNorm(dims[i], eps=1e-6, data_format="channels_first"),
                    nn.Conv2d(dims[i], dims[i+1], kernel_size=2, stride=2),
            )
            self.downsample_layers.append(downsample_layer)

        self.stages = nn.ModuleList() # 4 feature resolution stages, each consisting of multiple residual blocks
        dp_rates=[x.item() for x in torch.linspace(0, drop_path_rate, sum(depths))] 
        cur = 0
        for i in range(4):
            stage = nn.Sequential(
                *[Block(dim=dims[i], drop_path=dp_rates[cur + j], 
                layer_scale_init_value=layer_scale_init_value) for j in range(depths[i])]
            )
            self.stages.append(stage)
            cur += depths[i]

        self.norm = nn.LayerNorm(dims[-1], eps=1e-6) # final norm layer
        self.head = nn.Linear(dims[-1], num_classes)

        self.apply(self._init_weights)
        self.head.weight.data.mul_(head_init_scale)
        self.head.bias.data.mul_(head_init_scale)

    def _init_weights(self, m):
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            trunc_normal_(m.weight, std=.02)
            nn.init.constant_(m.bias, 0)

    def forward_features(self, x):
        for i in range(4):
            x = self.downsample_layers[i](x)
            x = self.stages[i](x)
        return self.norm(x.mean([-2, -1])) # global average pooling, (N, C, H, W) -> (N, C)

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x


In [ ]:
class Block(nn.Module):
    r""" ConvNeXt Block. There are two equivalent implementations:
    (1) DwConv -> LayerNorm (channels_first) -> 1x1 Conv -> GELU -> 1x1 Conv; all in (N, C, H, W)
    (2) DwConv -> Permute to (N, H, W, C); LayerNorm (channels_last) -> Linear -> GELU -> Linear; Permute back
    We use (2) as we find it slightly faster in PyTorch
    
    Args:
        dim (int): Number of input channels.
        drop_path (float): Stochastic depth rate. Default: 0.0
        layer_scale_init_value (float): Init value for Layer Scale. Default: 1e-6.
    """
    def __init__(self, dim, drop_path=0., layer_scale_init_value=1e-6):
        super().__init__()
        self.dwconv = nn.Conv2d(dim, dim, kernel_size=7, padding=3, groups=dim) # depthwise conv
        self.norm = LayerNorm(dim, eps=1e-6)
        self.pwconv1 = nn.Linear(dim, 4 * dim) # pointwise/1x1 convs, implemented with linear layers
        self.act = nn.GELU()
        self.pwconv2 = nn.Linear(4 * dim, dim)
        self.gamma = nn.Parameter(layer_scale_init_value * torch.ones((dim)), 
                                    requires_grad=True) if layer_scale_init_value > 0 else None
        self.drop_path = DropPath(drop_path) if drop_path > 0. else nn.Identity()

    def forward(self, x):
        input = x
        x = self.dwconv(x)
        x = x.permute(0, 2, 3, 1) # (N, C, H, W) -> (N, H, W, C)
        x = self.norm(x)
        x = self.pwconv1(x)
        x = self.act(x)
        x = self.pwconv2(x)
        if self.gamma is not None:
            x = self.gamma * x
        x = x.permute(0, 3, 1, 2) # (N, H, W, C) -> (N, C, H, W)

        x = input + self.drop_path(x)
        return x


In [ ]:
class LayerNorm(nn.Module):
    r""" LayerNorm that supports two data formats: channels_last (default) or channels_first. 
    The ordering of the dimensions in the inputs. channels_last corresponds to inputs with 
    shape (batch_size, height, width, channels) while channels_first corresponds to inputs 
    with shape (batch_size, channels, height, width).
    """
    def __init__(self, normalized_shape, eps=1e-6, data_format="channels_last"):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(normalized_shape))
        self.bias = nn.Parameter(torch.zeros(normalized_shape))
        self.eps = eps
        self.data_format = data_format
        if self.data_format not in ["channels_last", "channels_first"]:
            raise NotImplementedError 
        self.normalized_shape = (normalized_shape, )
    
    def forward(self, x):
        if self.data_format == "channels_last":
            return F.layer_norm(x, self.normalized_shape, self.weight, self.bias, self.eps)
        elif self.data_format == "channels_first":
            u = x.mean(1, keepdim=True)
            s = (x - u).pow(2).mean(1, keepdim=True)
            x = (x - u) / torch.sqrt(s + self.eps)
            x = self.weight[:, None, None] * x + self.bias[:, None, None]
            return x

In [ ]:
def convnext_tiny(pretrained=False,in_22k=False, **kwargs):
    model = ConvNeXt(depths=[3, 3, 9, 3], dims=[96, 192, 384, 768], **kwargs)
    if pretrained:
        url = model_urls['convnext_tiny_22k'] if in_22k else model_urls['convnext_tiny_1k']
        checkpoint = torch.hub.load_state_dict_from_url(url=url, map_location="cpu", check_hash=True)
        model.load_state_dict(checkpoint["model"])
    return model

In [ ]:
model_urls = {
    "convnext_tiny_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth",
    "convnext_small_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_1k_224_ema.pth",
    "convnext_base_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_1k_224_ema.pth",
    "convnext_large_1k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_1k_224_ema.pth",
    "convnext_tiny_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_22k_224.pth",
    "convnext_small_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth",
    "convnext_base_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_base_22k_224.pth",
    "convnext_large_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth",
    "convnext_xlarge_22k": "https://dl.fbaipublicfiles.com/convnext/convnext_xlarge_22k_224.pth",
}

In [ ]:
model = convnext_tiny(pretrained= False, in_22k= False)

In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters())

In [ ]:
pytorch_total_params 

28589128

In [ ]:
model.head = nn.Linear(768, 200)
# self.head = nn.Linear(dims[-1], num_classes)
model.head.weight.data.mul_(1)
model.head.bias.data.mul_(1)

tensor([-2.9429e-02, -2.6101e-02,  3.3958e-02,  1.5424e-02, -1.6329e-03,
        -3.4336e-02, -6.3030e-03,  6.0324e-03, -2.0140e-02,  1.4907e-02,
        -3.1033e-02, -8.9757e-03, -2.2465e-02, -2.7120e-02,  5.9825e-03,
        -6.0472e-03, -1.1577e-02,  4.9838e-03,  1.9827e-02,  3.4601e-02,
         1.2606e-02,  6.5946e-03, -1.4502e-02, -6.8717e-03, -2.3178e-02,
        -1.2911e-02, -9.4146e-03,  2.3769e-02, -2.7226e-02,  1.9225e-02,
         1.7618e-02, -3.4812e-04,  2.2362e-02, -2.2156e-02, -3.4127e-02,
        -3.3828e-02, -2.8084e-02,  2.4468e-03, -3.4594e-02,  2.7760e-02,
        -2.4601e-02,  1.6676e-04, -1.9516e-02, -1.2142e-02,  6.7058e-03,
         1.5028e-02, -3.5727e-02, -3.2827e-02, -3.2057e-02, -3.4269e-02,
         2.3051e-02,  2.6875e-02,  1.0378e-03, -3.1191e-02,  2.9113e-02,
        -5.3791e-03, -3.5473e-03, -2.0198e-02, -1.6058e-02,  8.2694e-03,
        -1.2081e-02,  9.4416e-05,  1.4879e-02, -3.3068e-02,  2.9786e-02,
        -1.8863e-02,  2.3542e-02, -3.1236e-02,  1.9

Train on the 448 1K pretrained model

In [ ]:
len_train = len(train_loader)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion, optimizer, scheduler, scaler = set_up(model, device, lr, weight_decay, len_train, epochs)

for epoch in range(epochs):
    train_accuracy, train_loss, learning_rate = train(model, device, batch_size, train_loader, optimizer, criterion, scheduler, scaler)
    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

    if not (epoch + 1) % 10 and epoch > 0:
        test_accuracy = evaluate(model, device, batch_size, test_loader, test_dataset)
        print("Test: {:.04f}%".format(test_accuracy))

Epoch 1/50: Train Acc 0.4654%, Train Loss 5.3045, Learning Rate 0.0300


Epoch 2/50: Train Acc 1.0638%, Train Loss 5.1345, Learning Rate 0.0299


Epoch 3/50: Train Acc 17.4867%, Train Loss 3.7166, Learning Rate 0.0297


Epoch 4/50: Train Acc 62.8158%, Train Loss 1.3654, Learning Rate 0.0295


Epoch 5/50: Train Acc 81.0505%, Train Loss 0.6689, Learning Rate 0.0293


...

Epoch 47/50: Train Acc 99.6343%, Train Loss 0.0011, Learning Rate 0.0003


Epoch 48/50: Train Acc 99.6343%, Train Loss 0.0012, Learning Rate 0.0001


Epoch 49/50: Train Acc 99.6343%, Train Loss 0.0011, Learning Rate 0.0000


Epoch 50/50: Train Acc 99.6343%, Train Loss 0.0012, Learning Rate 0.0000


Test: 83.3103%


Train on the 448 1K non-pretrained model

In [ ]:
# torch.cuda.empty_cache()
convnext_tiny = models.convnext_tiny(pretrained=False)
convnext_tiny.classifier[2] = nn.Linear(768, 200)
len_train = len(train_loader)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion, optimizer, scheduler, scaler = set_up(convnext_tiny, device, lr, weight_decay, len_train, epochs)

for epoch in range(epochs):
    train_accuracy, train_loss, learning_rate = train(convnext_tiny, device, batch_size, train_loader, optimizer, criterion, scheduler, scaler)
    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

    if not (epoch + 1) % 10 and epoch > 0:
        test_accuracy = evaluate(convnext_tiny, device, batch_size, test_loader, test_dataset)
        print("Test: {:.04f}%".format(test_accuracy))


Train:   0%|          | 0/188 [00:01<?, ?it/s, acc=3.1250%, loss=5.3674, lr=0.0300, num_correct=1]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1/50: Train Acc 0.7480%, Train Loss 5.5125, Learning Rate 0.0300


Epoch 2/50: Train Acc 0.6649%, Train Loss 5.2306, Learning Rate 0.0299


Epoch 3/50: Train Acc 1.1636%, Train Loss 5.1808, Learning Rate 0.0297


Epoch 4/50: Train Acc 0.9641%, Train Loss 5.1477, Learning Rate 0.0295


...

Epoch 47/50: Train Acc 2.6430%, Train Loss 4.9180, Learning Rate 0.0003


Epoch 48/50: Train Acc 2.6596%, Train Loss 4.9168, Learning Rate 0.0001


Epoch 49/50: Train Acc 2.7593%, Train Loss 4.9161, Learning Rate 0.0000


Epoch 50/50: Train Acc 2.8092%, Train Loss 4.9159, Learning Rate 0.0000


Test: 1.9676%


Train on the 224 1K non-pretrained model

In [ ]:
len_train = len(train_loader)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion, optimizer, scheduler, scaler = set_up(model, device, lr, weight_decay, len_train, epochs)

for epoch in range(epochs):
    train_accuracy, train_loss, learning_rate = train(model, device, batch_size, train_loader, optimizer, criterion, scheduler, scaler)
    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(epoch + 1, epochs, train_accuracy, train_loss, learning_rate))

    if not (epoch + 1) % 10 and epoch > 0:
        test_accuracy = evaluate(model, device, batch_size, test_loader, test_dataset)
        print("Test: {:.04f}%".format(test_accuracy))

Train:   0%|          | 0/94 [00:00<?, ?it/s, acc=0.0000%, loss=5.4352, lr=0.0300, num_correct=0]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch 1/50: Train Acc 0.7314%, Train Loss 5.9827, Learning Rate 0.0300


Epoch 2/50: Train Acc 1.0472%, Train Loss 5.2818, Learning Rate 0.0299


Epoch 3/50: Train Acc 1.3132%, Train Loss 5.1345, Learning Rate 0.0297


Epoch 4/50: Train Acc 1.4129%, Train Loss 5.0924, Learning Rate 0.0295


...

Epoch 47/50: Train Acc 2.8923%, Train Loss 4.9230, Learning Rate 0.0003


Epoch 48/50: Train Acc 2.6928%, Train Loss 4.9245, Learning Rate 0.0001


Epoch 49/50: Train Acc 2.6928%, Train Loss 4.9242, Learning Rate 0.0000


Epoch 50/50: Train Acc 2.7759%, Train Loss 4.9229, Learning Rate 0.0000


Test: 1.7777%


In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.save({
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'scheduler_state_dict' : scheduler.state_dict(),
        }, "/content/drive/MyDrive/FGVC-project"+"Model_"+str(epoch))